In [ ]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
from datetime import timedelta

from pylab import rcParams
rcParams['figure.figsize'] = 15, 10

import plotly.graph_objects as go
from plotly.subplots import make_subplots
from generate_results_utils import *
from datetime import timedelta, date, datetime

colors = ['#0080ff', '#009900', "#ff9933", "#911eb4", "#46f0f0", "#f032e6", "#d2f53c", "#fabebe", "#008080", 
          "#e6beff", "#aa6e28", "#fffac8", "#800000", "#aaffc3", "#808000", "#ffd8b1", "#000080", "#808080", "#ffffff", "#000000", "#e6194b", "#3cb44b", "#ffe119", "#0082c8", "#f58231"]
ensemble_colors = ['#000000', '#0080ff', '#009900', "#ff9933", "#911eb4", "#46f0f0", "#f032e6", "#d2f53c", "#fabebe", "#008080", 
          "#e6beff", "#aa6e28", "#fffac8", "#800000", "#aaffc3", "#808000", "#ffd8b1", "#000080", "#808080", "#ffffff", "#000000", "#e6194b", "#3cb44b", "#ffe119", "#0082c8", "#f58231"]

start_train = datetime.strptime("22.06.2022", "%d.%m.%Y") 
end_train = start_train + timedelta(days=27) # , hours=23, minutes=59, seconds=59)
split_dates = []
for i in range(5):
    split_dates.append(end_train)
    end_train += timedelta(days=28)

In [ ]:
split_dates

In [ ]:
actions_suffix = '_actions.csv'
value_suffix = '_state.csv'

In [ ]:
def get_dfs(names, model, stat_files):
    dfs = []
    for idx, name in enumerate(names):
        strategy_name = f'{model}/{stat_files[idx]}/all'
        portfolio_prefix = f'../results/cs_eval/{strategy_name}'

        df = pd.read_csv(f"{portfolio_prefix}{value_suffix}")  # , index_col=0
        df = df.rename(columns={df.columns[0]: 'date'})
        df.loc[0, 'date'] = pd.to_datetime(df.loc[1]['date']) - timedelta(days=1)
        df['date'] = pd.to_datetime(df['date'])
        df['return'] = df['account_value'].pct_change() + 1
        df['equity'] = (df['return']).cumprod()
        df.loc[0, 'return'] = 1
        df.loc[0, 'equity'] = 1
        df.index = df['date']
        df = df.drop('date', axis=1)

        df2 = pd.read_csv(f"{portfolio_prefix}{actions_suffix}", index_col=0)
        df2.index = pd.to_datetime(df2.index)

        df['strategy'] = df['return']
        df_strategy = df[['strategy']]         
        df_returns = df_strategy.cumprod()
        dfs.append(df_returns)
    return dfs

def get_ensemble(stat_files):
    dfs = []
    for fname in stat_files:
        portfolio_prefix = f'../results/ce/{fname}/all'

        df = pd.read_csv(f"{portfolio_prefix}{value_suffix}")  # , index_col=0
        df = df.rename(columns={df.columns[0]: 'date'})
        df.loc[0, 'date'] = pd.to_datetime(df.loc[1]['date']) - timedelta(days=1)
        df['date'] = pd.to_datetime(df['date'])
        df['return'] = df['account_value'].pct_change() + 1
        df['equity'] = (df['return']).cumprod()
        df.loc[0, 'return'] = 1
        df.loc[0, 'equity'] = 1
        df.index = df['date']
        df = df.drop('date', axis=1)

        # df2 = pd.read_csv(f"{portfolio_prefix}{actions_suffix}", index_col=0)
        # df2.index = pd.to_datetime(df2.index)

        df['strategy'] = df['return']
        df_strategy = df[['strategy']]         
        df_returns = df_strategy.cumprod()
        dfs.append(df_returns)
    return dfs

def get_benchmark(dfs):
    benchmark_returns_1d = '../datasets/thesis/benchmark/1d_benchmark_returns.csv'
    df_benchmark = pd.read_csv(f"{benchmark_returns_1d}")
    df_benchmark.index = pd.to_datetime(df_benchmark['date'])
    df_benchmark = df_benchmark.drop('date', axis=1)
    df_benchmark = df_benchmark[(df_benchmark.index >= dfs[0].index[0]) & (df_benchmark.index <= dfs[0].index[-1])]
    df_benchmark = df_benchmark.cumprod()
    return df_benchmark

def get_prices(sample_strat):
    stock_prices = '../datasets/thesis/1d/all_1d.csv'
    df3 = pd.read_csv(stock_prices)
    df3['date'] = pd.to_datetime(df3['date'])
    # get or column with close price for each stock per tick
    df3 = df3.pivot(index="date", columns="tic", values="close")
    # limit dataset to portfolio range
    df3 = df3[(df3.index >= sample_strat.index[0]) & (df3.index <= sample_strat.index[-1])]
    # get relative change from price
    df3 = df3.pct_change()
    # set first row to 0
    df3.loc[df3.iloc[0].name] = 0
    # get cumulative return for each stock
    df3 = (df3 + 1).cumprod()
    return df3

def plot_performance(model, names, strats, benchmarks, prices, plot_folder, show_prices=True, save=True, split_dates=[], with_ensemble=False, colors=colors, show=True):    
    fig = make_subplots(specs=[[{"secondary_y": True}]])
        
    if show_prices:
        for idx, name in enumerate(prices.columns):
            fig.add_trace(go.Scatter(x=prices.index,y=prices[name],name=name, showlegend=False, marker={'color':"#e0e0e0", 'dash':'dot'}, opacity=0.8),secondary_y=False)

    for idx, name in enumerate(names):
        cdf = strats[idx]
        fig.add_trace(go.Scatter(x=cdf.index,y=cdf['strategy'],name=name, marker_color=colors[idx]),secondary_y=False)

    for i, split in enumerate(split_dates):
        fig.add_vline(x=split, line_width=2, line_color='#ccc', opacity=0.7)
        fig.update_xaxes(showgrid=False)
        
    # benchmarks = dfs[0][["CRP", "BAH", "MPT"]]
    cs = ['#ff0000', '#8A9A5B', '#36454F', '#A9A9A9']
    # for idx, name in enumerate(benchmarks.columns):
    fig.add_trace(go.Scatter(x=benchmarks.index,y=benchmarks['BAH'],name='BAH', marker={'color':cs[0]}, line={'dash':'dashdot'}),secondary_y=False)
   
    fig.update_layout(height=400, template='plotly_white') # title_text=f'{model} Strategy Performance compared to benchmarks', 
    if show:
        fig.show()
    if save:
        filename_prefix = f"{plot_folder}/performance_compare_{model}"
        fig.write_html(f"{filename_prefix}.html")
        fig.write_image(f"{filename_prefix}.png", width=1200, scale=2)            

## Old Plots

In [ ]:
# names = ["V213", "V219", "V221"]
# model = "A2C"
# stat_files = ["EVAL_A2C_V213_12201326_200M", "EVAL_A2C_V219_12201317_200M", "EVAL_A2C_V221_12200914_200M"]

# dfs_a2c = get_dfs(names, model, stat_files)
# df_benchmark = get_benchmark(dfs_a2c)
# df_prices = get_prices(dfs_a2c[0])

# plot_folder = f'../results/cs_eval/{model}'
# plot_performance(model, names, dfs_a2c, df_benchmark, df_prices, plot_folder)

In [ ]:
# names = ["V205", "V207", "V208"]
# model = "PPO"
# stat_files = ["EVAL_PPO_V205_12201259_120M", "EVAL_PPO_V207_12200935_120M", "EVAL_PPO_V208_12200925_120M"]

# dfs_ppo = get_dfs(names, model, stat_files)
# df_benchmark = get_benchmark(dfs_ppo)
# df_prices = get_prices(dfs_ppo[0])

# plot_folder = f'../results/cs_eval/{model}'
# plot_performance(model, names, dfs_ppo, df_benchmark, df_prices, plot_folder)

In [ ]:
# names = ["V203", "V213", "V214"]
# model = "TD3"
# stat_files = ["EVAL_TD3_V203_12201307_12M", "EVAL_TD3_V213_12201319_12M", "EVAL_TD3_V214_12201313_12M"]

# dfs_td3 = get_dfs(names, model, stat_files)
# df_benchmark = get_benchmark(dfs_td3)
# df_prices = get_prices(dfs_td3[0])

# plot_folder = f'../results/cs_eval/{model}'
# plot_performance(model, names, dfs_td3, df_benchmark, df_prices, plot_folder)

In [ ]:
# best = ["EVAL_A2C_V221_12200914_200M", "EVAL_PPO_V208_12200925_120M", "EVAL_TD3_V214_12201313_12M"]
# dfs2 = []
# dfs2.append(dfs_a2c[2])
# dfs2.append(dfs_ppo[2])
# dfs2.append(dfs_td3[2])
# names = ["A2C", "PPO", "TD3"]
# df_benchmark = get_benchmark(dfs2)
# df_prices = get_prices(dfs2[0])

# plot_folder = f'../results/cs_eval/'
# model = "1D"
# plot_performance(model, names, dfs2, df_benchmark, df_prices, plot_folder, show_prices=False)


In [ ]:
# cfg = {
#     "A2C": ["EVAL_A2C_V221_12200914_200M"],
#     "PPO": ["EVAL_PPO_V208_01121911_1D_180M"],
#     "TD3": ["EVAL_TD3_V214_12201313_12M"],    
# }

# dfs2 = []
# dfs_a2c = get_dfs(["A2C"], "A2C", cfg["A2C"])
# dfs_ppo = get_dfs(["PPO"], "PPO", cfg["PPO"])
# dfs_td3 = get_dfs(["TD3"], "TD3", cfg["TD3"])


# dfs2.append(dfs_a2c[0])
# dfs2.append(dfs_ppo[0])
# dfs2.append(dfs_td3[0])
# names = ["A2C", "PPO", "TD3"]
# df_benchmark = get_benchmark(dfs2)
# df_prices = get_prices(dfs2[0])

# plot_folder = f'../results/cs_eval/'
# model = "1D"
# plot_performance(model, names, dfs2, df_benchmark, df_prices, plot_folder, show_prices=False, save=True)


In [ ]:
# cfg = {
#     "A2C": ["EVAL_A2C_V221_01121916_12H_600M"],
#     "PPO": ["EVAL_PPO_V208_01121435_12H_600M"],
#     "TD3": ["EVAL_TD3_V214_01130958_12H_54M"],    
# }

# dfs2 = []
# dfs_a2c = get_dfs(["A2C"], "A2C", cfg["A2C"])
# dfs_ppo = get_dfs(["PPO"], "PPO", cfg["PPO"])
# dfs_td3 = get_dfs(["TD3"], "TD3", cfg["TD3"])


# dfs2.append(dfs_a2c[0])
# dfs2.append(dfs_ppo[0])
# dfs2.append(dfs_td3[0])
# names = ["A2C", "PPO", "TD3"]
# df_benchmark = get_benchmark(dfs2)
# df_prices = get_prices(dfs2[0])

# plot_folder = f'../results/cs_eval/'
# model = "12H"
# plot_performance(model, names, dfs2, df_benchmark, df_prices, plot_folder, show_prices=False, save=True)


In [ ]:
# cfg = {
#     "A2C": ["EVAL_A2C_V221_01121914_6H_550M"],
#     "PPO": ["EVAL_PPO_V208_01121514_6H_550M"],
#     "TD3": ["EVAL_TD3_V214_01130933_6H_54M"],    
# }

# dfs2 = []
# dfs_a2c = get_dfs(["A2C"], "A2C", cfg["A2C"])
# dfs_ppo = get_dfs(["PPO"], "PPO", cfg["PPO"])
# dfs_td3 = get_dfs(["TD3"], "TD3", cfg["TD3"])


# dfs2.append(dfs_a2c[0])
# dfs2.append(dfs_ppo[0])
# dfs2.append(dfs_td3[0])
# names = ["A2C", "PPO", "TD3"]
# df_benchmark = get_benchmark(dfs2)
# df_prices = get_prices(dfs2[0])

# plot_folder = f'../results/cs_eval/'
# model = "6H"
# plot_performance(model, names, dfs2, df_benchmark, df_prices, plot_folder, show_prices=False, save=True)


In [ ]:
# cfg = {
#     "A2C": ["EVAL_A2C_V221_01122308_1H_240M"],
#     "PPO": ["EVAL_PPO_V208_01121518_1H_200M"],
#     "TD3": ["EVAL_TD3_V214_01122319_1H_42M"],    
# }

# dfs2 = []
# dfs_a2c = get_dfs(["A2C"], "A2C", cfg["A2C"])
# dfs_ppo = get_dfs(["PPO"], "PPO", cfg["PPO"])
# dfs_td3 = get_dfs(["TD3"], "TD3", cfg["TD3"])


# dfs2.append(dfs_a2c[0])
# dfs2.append(dfs_ppo[0])
# dfs2.append(dfs_td3[0])
# names = ["A2C", "PPO", "TD3"]
# df_benchmark = get_benchmark(dfs2)
# df_prices = get_prices(dfs2[0])

# plot_folder = f'../results/cs_eval/'
# model = "1H"
# plot_performance(model, names, dfs2, df_benchmark, df_prices, plot_folder, show_prices=False, save=True)


In [ ]:
# cfg = {
#     "A2C": ["EVAL_A2C_V221_01122308_1H_240M", "EVAL_A2C_V221_01121914_6H_550M", "EVAL_A2C_V221_01121916_12H_600M", "EVAL_A2C_V221_12200914_200M"]
# }

# names = ["1H", "6H", "12H", "1D"]
# dfs = get_dfs(names, "A2C", cfg["A2C"])


# df_benchmark = get_benchmark(dfs)
# df_prices = get_prices(dfs[0])

# plot_folder = f'../results/cs_eval/'
# model = "All A2C"
# plot_performance(model, names, dfs, df_benchmark, df_prices, plot_folder, show_prices=False, save=True)


In [ ]:
# cfg = {
#     "PPO": ["EVAL_PPO_V208_01121518_1H_200M", "EVAL_PPO_V208_01121514_6H_550M", "EVAL_PPO_V208_01121435_12H_600M", "EVAL_PPO_V208_01121911_1D_180M"]
# }

# names = ["1H", "6H", "12H", "1D"]
# dfs = get_dfs(names, "PPO", cfg["PPO"])


# df_benchmark = get_benchmark(dfs)
# df_prices = get_prices(dfs[0])

# plot_folder = f'../results/cs_eval/'
# model = "All PPO"
# plot_performance(model, names, dfs, df_benchmark, df_prices, plot_folder, show_prices=False, save=True)

In [ ]:
# cfg = {
#     "TD3": ["EVAL_TD3_V214_01122319_1H_42M", "EVAL_TD3_V214_01130933_6H_54M", "EVAL_TD3_V214_01130958_12H_54M", "EVAL_TD3_V214_12201313_12M"]
# }

# names = ["1H", "6H", "12H", "1D"]
# dfs = get_dfs(names, "TD3", cfg["TD3"])


# df_benchmark = get_benchmark(dfs)
# df_prices = get_prices(dfs[0])

# plot_folder = f'../results/cs_eval/'
# model = "All TD3"
# plot_performance(model, names, dfs, df_benchmark, df_prices, plot_folder, show_prices=False, save=True)

## New Plots

# Split by Time Res

In [ ]:
cfg = {
    "A2C": ["EVAL_A2C_V221_2301221555_1D_mini_200"],
    "PPO": ["EVAL_PPO_V208_2301180901_1D_short"],
    "TD3": ["EVAL_TD3_V214_2301180652_1D_short"],    
}

dfs2 = []
dfs_a2c = get_dfs(["A2C"], "A2C", cfg["A2C"])
dfs_ppo = get_dfs(["PPO"], "PPO", cfg["PPO"])
dfs_td3 = get_dfs(["TD3"], "TD3", cfg["TD3"])


dfs2.append(dfs_a2c[0])
dfs2.append(dfs_ppo[0])
dfs2.append(dfs_td3[0])
names = ["A2C (24h)", "PPO (24h)", "TD3 (24h)"]
df_benchmark = get_benchmark(dfs2)
df_prices = get_prices(dfs2[0])

plot_folder = f'../results/cs_eval/'
model = "1D"
plot_performance(model, names, dfs2, df_benchmark, df_prices, plot_folder, show_prices=False, save=True, split_dates=split_dates)



In [ ]:
cfg = {
    "A2C": ["EVAL_A2C_V221_2301171652_12H_short"],
    "PPO": ["EVAL_PPO_V208_2301171700_12H_short"],
    "TD3": ["EVAL_TD3_V214_2301171757_12H_short"],    
}

dfs2 = []
dfs_a2c = get_dfs(["A2C"], "A2C", cfg["A2C"])
dfs_ppo = get_dfs(["PPO"], "PPO", cfg["PPO"])
dfs_td3 = get_dfs(["TD3"], "TD3", cfg["TD3"])


dfs2.append(dfs_a2c[0])
dfs2.append(dfs_ppo[0])
dfs2.append(dfs_td3[0])
names = ["A2C (12h)", "PPO (12h)", "TD3 (12h)"]
df_benchmark = get_benchmark(dfs2)
df_prices = get_prices(dfs2[0])

plot_folder = f'../results/cs_eval/'
model = "12H"
plot_performance(model, names, dfs2, df_benchmark, df_prices, plot_folder, show_prices=False, save=True, split_dates=split_dates)


In [ ]:
cfg = {
    "A2C": ["EVAL_A2C_V221_2301172001_6H_short"],
    "PPO": ["EVAL_PPO_V208_2301171958_6H_short"],
    "TD3": ["EVAL_TD3_V214_2301172004_6H_short"],    
}

dfs2 = []
dfs_a2c = get_dfs(["A2C"], "A2C", cfg["A2C"])
dfs_ppo = get_dfs(["PPO"], "PPO", cfg["PPO"])
dfs_td3 = get_dfs(["TD3"], "TD3", cfg["TD3"])


dfs2.append(dfs_a2c[0])
dfs2.append(dfs_ppo[0])
dfs2.append(dfs_td3[0])
names = ["A2C (6h)", "PPO (6h)", "TD3 (6h)"]
df_benchmark = get_benchmark(dfs2)
df_prices = get_prices(dfs2[0])

plot_folder = f'../results/cs_eval/'
model = "6H"
plot_performance(model, names, dfs2, df_benchmark, df_prices, plot_folder, show_prices=False, save=True, split_dates=split_dates)


In [ ]:
cfg = {
    "A2C": ["EVAL_A2C_V221_2301172318_1H_short"],
    "PPO": ["EVAL_PPO_V208_2301172325_1H_short"],
    "TD3": ["EVAL_TD3_V214_2301172224_1H_short"],    
}

dfs2 = []
dfs_a2c = get_dfs(["A2C"], "A2C", cfg["A2C"])
dfs_ppo = get_dfs(["PPO"], "PPO", cfg["PPO"])
dfs_td3 = get_dfs(["TD3"], "TD3", cfg["TD3"])


dfs2.append(dfs_a2c[0])
dfs2.append(dfs_ppo[0])
dfs2.append(dfs_td3[0])
names = ["A2C (1h)", "PPO (1h)", "TD3 (1h)"]
df_benchmark = get_benchmark(dfs2)
df_prices = get_prices(dfs2[0])

plot_folder = f'../results/cs_eval/'
model = "1H"
plot_performance(model, names, dfs2, df_benchmark, df_prices, plot_folder, show_prices=False, save=True, split_dates=split_dates)


In [ ]:
# plot_performance("Benchmarks", [], [], df_benchmark, df_prices, plot_folder, show_prices=False, save=True)

# Split By Algorithm

In [ ]:
cfg = {
    "TD3": ["EVAL_TD3_V214_2301172224_1H_short", "EVAL_TD3_V214_2301172004_6H_short", "EVAL_TD3_V214_2301171757_12H_short", "EVAL_TD3_V214_2301180652_1D_short"],
    "PPO": ["EVAL_PPO_V208_2301172325_1H_short", "EVAL_PPO_V208_2301171958_6H_short", "EVAL_PPO_V208_2301171700_12H_short", "EVAL_PPO_V208_2301180901_1D_short"],
    "A2C": ["EVAL_A2C_V221_2301172318_1H_short", "EVAL_A2C_V221_2301172001_6H_short", "EVAL_A2C_V221_2301171652_12H_short", "EVAL_A2C_V221_2301221555_1D_mini_200"],
    "res": ["1h", "6h", "12h", "1d"]
}

In [ ]:
names = ["TD3 (1h)", "TD3 (6h)", "TD3 (12h)", "TD3 (24h)"]
dfs = get_dfs(names, "TD3", cfg["TD3"])

df_benchmark = get_benchmark(dfs)
df_prices = get_prices(dfs[0])

plot_folder = f'../results/cs_eval/'
model = "TD3"
plot_performance(model, names, dfs, df_benchmark, df_prices, plot_folder, show_prices=False, save=True, split_dates=split_dates)

In [ ]:
names = ["PPO (1h)", "PPO (6h)", "PPO (12h)", "PPO (24h)"]
dfs = get_dfs(names, "PPO", cfg["PPO"])

df_benchmark = get_benchmark(dfs)
df_prices = get_prices(dfs[0])

plot_folder = f'../results/cs_eval/'
model = "PPO"
plot_performance(model, names, dfs, df_benchmark, df_prices, plot_folder, show_prices=False, save=True, split_dates=split_dates)

In [ ]:
names = ["A2C (1h)", "A2C (6h)", "A2C (12h)", "A2C (24h)"]
dfs = get_dfs(names, "A2C", cfg["A2C"])

df_benchmark = get_benchmark(dfs)
df_prices = get_prices(dfs[0])

plot_folder = f'../results/cs_eval/'
model = "A2C"
plot_performance(model, names, dfs, df_benchmark, df_prices, plot_folder, show_prices=False, save=True, split_dates=split_dates)

# Individual Plots

In [ ]:
cfg = {
    "TD3": ["EVAL_TD3_V214_2301172224_1H_short", "EVAL_TD3_V214_2301172004_6H_short", "EVAL_TD3_V214_2301171757_12H_short", "EVAL_TD3_V214_2301180652_1D_short"],
    "PPO": ["EVAL_PPO_V208_2301172325_1H_short", "EVAL_PPO_V208_2301171958_6H_short", "EVAL_PPO_V208_2301171700_12H_short", "EVAL_PPO_V208_2301180901_1D_short"],
    "A2C": ["EVAL_A2C_V221_2301172318_1H_short", "EVAL_A2C_V221_2301172001_6H_short", "EVAL_A2C_V221_2301171652_12H_short", "EVAL_A2C_V221_2301221555_1D_mini_200"],
    "res": ["1h", "6h", "12h", "1d"]
}

In [ ]:
algs = ["A2C", "PPO", "TD3"]
plot_folder = f'../results/cs_eval/'
for alg in algs:
    for idx, res in enumerate(cfg['res']):
        my_cfg = [cfg[alg][idx]]
        names = [f'{alg} ({res})']
        model = f"single_{alg}_{res}"     
        print(alg, res, cfg[alg][idx], names, my_cfg, model)   
        dfs = get_dfs(names, alg, my_cfg)
        df_benchmark = get_benchmark(dfs)
        df_prices = get_prices(dfs[0])
        plot_performance(model, names, dfs, df_benchmark, df_prices, plot_folder, show_prices=False, save=True, split_dates=split_dates, colors=ensemble_colors, show=False)

# Weights

In [ ]:
def get_weights(run_dir, run_name, res, root_dir='..', file_prefix='all'):
    results_dir = f"{root_dir}/results/{run_dir}/{run_name}"
    results_file_prefix = f"{results_dir}/{file_prefix}"
    filename = f"{results_file_prefix}_state.csv"
    
    results_account_value = pd.read_csv(filename)
    ratios = get_asset_ratios(results_account_value, res)
    ratio_values = convert_ratios_to_value(ratios, results_account_value)
    
    return ratios, ratio_values

def generate_weights_for_run(run_dir, run_name, res, root_dir='..', file_prefix='all', save=True, show=False, show_both=False):
    
    ratios, ratios_full = get_weights(run_dir, run_name, res, root_dir, file_prefix)    
    
    if save:
        weights_norm_file = f"{results_dir}/weights_normalized.png"
        weights_file = f"{results_dir}/weights.png"
    else:
        weights_norm_file, weights_file = "", ""
    
    print_weights(ratios, save_location=weights_norm_file, show=(show and show_both))
    print_weights(ratios_full, save_location=weights_file, show=show)
    
def generate_weights_plot(ratio, names, colors, filename_prefix, show=False, split_dates=[]):
    fig = go.Figure()
    x = ratio['date']
    for idx, name in enumerate(names):
        y = ratio[name]
        # fill = 'tonexty' if idx > 0 else 'tozeroy'
        fig.add_trace(go.Scatter(x=x, y=y, hoverinfo='x+y', mode='lines', name=name.replace('USDT', ''), line=dict(width=0.5, color=colors[idx]), stackgroup='one', legendrank=10-idx))
    fig.update_layout(height=400, template='plotly_white') # title_text=f'Weights', 
    for i, split in enumerate(split_dates):
        fig.add_vline(x=split, line_width=2, line_color='#ccc', opacity=0.7)
        fig.update_xaxes(showgrid=False)
    if show:
        fig.show()
    if filename_prefix != "":
        fig.write_html(f"{filename_prefix}/weights.html")
        fig.write_image(f"{filename_prefix}/weights.png", width=1200, scale=1) 

In [ ]:
# generate_weights_for_run('cs_eval', 'A2C/EVAL_A2C_V221_2301221555_1D_mini_200', '1d', show=True, save=False)

In [ ]:
alg = 'A2C'
for idx, run in enumerate(cfg[alg]):
    print(run, cfg['res'][idx])
    ratios, ratio_values = get_weights('cs_eval', f'{alg}/{run}', cfg['res'][idx])
    generate_weights_plot(ratio_values, crypto_names, weight_colors, f"../results/cs_eval/{alg}/{run}", split_dates=split_dates)

In [ ]:
names

In [ ]:
alg = 'PPO'
for idx, run in enumerate(cfg[alg]):
    print(run, cfg['res'][idx])
    ratios, ratio_values = get_weights('cs_eval', f'{alg}/{run}', cfg['res'][idx])
    generate_weights_plot(ratio_values, crypto_names, weight_colors, f"../results/cs_eval/{alg}/{run}", split_dates=split_dates, show=False) 

In [ ]:
alg = 'TD3'
for idx, run in enumerate(cfg[alg]):
    print(run, cfg['res'][idx])
    ratios, ratio_values = get_weights('cs_eval', f'{alg}/{run}', cfg['res'][idx])
    generate_weights_plot(ratio_values, crypto_names, weight_colors, f"../results/cs_eval/{alg}/{run}", split_dates=split_dates, show=False) 

In [ ]:
generate_weights_plot(ratio_values, names, weight_colors, f"../results/cs_eval/{alg}/{run}")

# Ensemble + Singles

In [ ]:
cfg = {
    "TD3": ["EVAL_TD3_V214_2301172224_1H_short", "EVAL_TD3_V214_2301172004_6H_short", "EVAL_TD3_V214_2301171757_12H_short", "EVAL_TD3_V214_2301180652_1D_short"],
    "PPO": ["EVAL_PPO_V208_2301172325_1H_short", "EVAL_PPO_V208_2301171958_6H_short", "EVAL_PPO_V208_2301171700_12H_short", "EVAL_PPO_V208_2301180901_1D_short"],
    "A2C": ["EVAL_A2C_V221_2301172318_1H_short", "EVAL_A2C_V221_2301172001_6H_short", "EVAL_A2C_V221_2301171652_12H_short", "EVAL_A2C_V221_2301221555_1D_mini_200"],
    "ensemble": ["ce_1h_2301141443_1", "ce_6h_2301132259_1", "ce_12h_2301132144_1", "ce_1d_2301221639_fix"],
    "res": ["1h", "6h", "12h", "1d"]
}

In [ ]:
my_cfg = {
    "A2C": [cfg['A2C'][3]],
    "PPO": [cfg['PPO'][3]],
    "TD3": [cfg['TD3'][3]],
    "ensemble": [cfg['ensemble'][3]],
}

dfs2 = []
dfs_a2c = get_dfs(["A2C"], "A2C", my_cfg["A2C"])
dfs_ppo = get_dfs(["PPO"], "PPO", my_cfg["PPO"])
dfs_td3 = get_dfs(["TD3"], "TD3", my_cfg["TD3"])
dfs_ensemble = get_ensemble(my_cfg['ensemble'])

dfs2.append(dfs_ensemble[0])
dfs2.append(dfs_a2c[0])
dfs2.append(dfs_ppo[0])
dfs2.append(dfs_td3[0])

names = ["Ensemble (24h)", "A2C (24h)", "PPO (24h)", "TD3 (24h)"]
df_benchmark = get_benchmark(dfs2)
df_prices = get_prices(dfs2[0])

plot_folder = f'../results/ce/'
model = "ALL 1D"
plot_performance(model, names, dfs2, df_benchmark, df_prices, plot_folder, show_prices=False, save=True, split_dates=split_dates, colors=ensemble_colors)

In [ ]:
my_cfg = {
    "A2C": [cfg['A2C'][2]],
    "PPO": [cfg['PPO'][2]],
    "TD3": [cfg['TD3'][2]],
    "ensemble": [cfg['ensemble'][2]],
}

dfs2 = []
dfs_a2c = get_dfs(["A2C"], "A2C", my_cfg["A2C"])
dfs_ppo = get_dfs(["PPO"], "PPO", my_cfg["PPO"])
dfs_td3 = get_dfs(["TD3"], "TD3", my_cfg["TD3"])
dfs_ensemble = get_ensemble(my_cfg['ensemble'])

dfs2.append(dfs_ensemble[0])
dfs2.append(dfs_a2c[0])
dfs2.append(dfs_ppo[0])
dfs2.append(dfs_td3[0])

names = ["Ensemble (12h)", "A2C (12h)", "PPO (12h)", "TD3 (12h)"]
df_benchmark = get_benchmark(dfs2)
df_prices = get_prices(dfs2[0])

plot_folder = f'../results/ce/'
model = "ALL 12H"
plot_performance(model, names, dfs2, df_benchmark, df_prices, plot_folder, show_prices=False, save=True, split_dates=split_dates, colors=ensemble_colors)

In [ ]:
my_cfg = {
    "A2C": [cfg['A2C'][1]],
    "PPO": [cfg['PPO'][1]],
    "TD3": [cfg['TD3'][1]],
    "ensemble": [cfg['ensemble'][1]],
}

dfs2 = []
dfs_a2c = get_dfs(["A2C"], "A2C", my_cfg["A2C"])
dfs_ppo = get_dfs(["PPO"], "PPO", my_cfg["PPO"])
dfs_td3 = get_dfs(["TD3"], "TD3", my_cfg["TD3"])
dfs_ensemble = get_ensemble(my_cfg['ensemble'])

dfs2.append(dfs_ensemble[0])
dfs2.append(dfs_a2c[0])
dfs2.append(dfs_ppo[0])
dfs2.append(dfs_td3[0])

names = ["Ensemble (6h)", "A2C (6h)", "PPO (6h)", "TD3 (6h)"]
df_benchmark = get_benchmark(dfs2)
df_prices = get_prices(dfs2[0])

plot_folder = f'../results/ce/'
model = "ALL 6H"
plot_performance(model, names, dfs2, df_benchmark, df_prices, plot_folder, show_prices=False, save=True, split_dates=split_dates, colors=ensemble_colors)

In [ ]:
my_cfg = {
    "A2C": [cfg['A2C'][0]],
    "PPO": [cfg['PPO'][0]],
    "TD3": [cfg['TD3'][0]],
    "ensemble": [cfg['ensemble'][0]],
}

dfs2 = []
dfs_a2c = get_dfs(["A2C"], "A2C", my_cfg["A2C"])
dfs_ppo = get_dfs(["PPO"], "PPO", my_cfg["PPO"])
dfs_td3 = get_dfs(["TD3"], "TD3", my_cfg["TD3"])
dfs_ensemble = get_ensemble(my_cfg['ensemble'])

dfs2.append(dfs_ensemble[0])
dfs2.append(dfs_a2c[0])
dfs2.append(dfs_ppo[0])
dfs2.append(dfs_td3[0])

names = ["Ensemble (1h)", "A2C (1h)", "PPO (1h)", "TD3 (1h)"]
df_benchmark = get_benchmark(dfs2)
df_prices = get_prices(dfs2[0])

plot_folder = f'../results/ce/'
model = "ALL 1H"
plot_performance(model, names, dfs2, df_benchmark, df_prices, plot_folder, show_prices=False, save=True, split_dates=split_dates, colors=ensemble_colors)